# Car Price Prediction API using FastAPI
This notebook builds and serves a machine learning model to predict car prices via an API using FastAPI

In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [2]:
# Load the dataset

df=pd.read_csv("cars_v1.csv", encoding="ISO-8859-1")

# Show first few rows
df.head()

,Make,Model,Price,Type,ABS,BootSpace,GearType,AirBag,Engine,FuelCapacity,Mileage
0,Ashok Leyland Stile,Ashok Leyland Stile LE 8-STR (Diesel),750,MPV,No,500.0,Manual,No,1461.0,50.0,20.7
1,Ashok Leyland Stile,Ashok Leyland Stile LS 8-STR (Diesel),800,MPV,No,500.0,Manual,No,1461.0,50.0,20.7
2,Ashok Leyland Stile,Ashok Leyland Stile LX 8-STR (Diesel),830,MPV,No,500.0,Manual,No,1461.0,50.0,20.7
3,Ashok Leyland Stile,Ashok Leyland Stile LS 7-STR (Diesel),850,MPV,No,500.0,Manual,No,1461.0,50.0,20.7
4,Ashok Leyland Stile,Ashok Leyland Stile LS 7-STR Alloy (Diesel),880,MPV,No,500.0,Manual,No,1461.0,50.0,20.7


In [3]:
# Drop columns not needed
df.drop(columns=["Model"], inplace=True)


In [4]:
# Check for missing values
df.isnull().sum()

Make              0
Price             0
Type              0
ABS              16
BootSpace       179
GearType         16
AirBag           21
Engine            7
FuelCapacity      0
Mileage         171
dtype: int64

In [5]:
# Split features and Target
x = df.drop(columns=["Price"])
y = df["Price"]

In [6]:
# Define column types
numeric_features = ["BootSpace","Engine","FuelCapacity","Mileage"]
categorical_features = ["Make","Type","ABS","GearType","AirBag"]

In [7]:
# Create Preprocessing Pipelines
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")), 
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])

In [8]:
# Create full pipeline with Model
model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", RandomForestRegressor(random_state=42))
])

In [9]:
# Train-Test split and train model
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2, random_state=42)

# Fit the model
model.fit(x_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['BootSpace', 'Engine',
                                                   'FuelCapacity', 'Mileage']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Make', 'Type', 'ABS',
                                                   'GearType', 'AirBag'])])),
                ('regressor', RandomForestRegressor(random_state=42))])

In [10]:
# Evaluate model
y_pred = model.predict(x_test)
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
print("MAE:", mean_absolute_error(y_test, y_pred))

RMSE: 3005.9356845593406
MAE: 858.6476623247012


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [11]:
# Save the model
joblib.dump(model,"car_price_predictor.pkl")

['car_price_predictor.pkl']

In [12]:
# Predict on mew data
new_data = pd.DataFrame([{
    "Make": "Toyota",
    "Type": "Sedan",
    "ABS": "Yes",
    "BootSpace": 400,
    "GearType": "Manual",
    "AirBag": "Yes",
    "Engine": 1500,
    "FuelCapacity":40,
    "Mileage":18
}])

loaded_model=joblib.load("car_price_predictor.pkl")
loaded_model.predict(new_data)
                

array([700.45316667])

In [13]:
%%writefile car_price_api.py

from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd
import joblib

# Load the model
model = joblib.load("car_price_predictor.pkl")

class CarFeatures(BaseModel):
    Make: str
    Type: str
    ABS: str
    BootSpace: float | None=None
    GearType: str
    AirBag: str
    Engine: float  | None=None
    FuelCapacity: float
    Mileage: float  | None=None

app = FastAPI()

@app.get("/")
def home():
    return{"message": "Welcome to Car Price Predictio API"}

@app.post("/predict")
def predict_price(features: CarFeatures):
    data = pd.DataFrame([features.dict()])
    prediction = model.predict(data)[0]
    return{"predicted_price":round(prediction,2)}

Overwriting car_price_api.py


##  How to run the API
1. In terminal or Anaconda prompt:
   uvicorn car_price_api:app --reload
2. Open the browser and go to http://127.0.0.1:8000/docs
3. Provide the data to be tested in the predict space.